In [26]:
from keras.layers import BatchNormalization,  Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from keras.layers import Input, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from keras.applications.vgg16 import VGG16
from keras.layers.core import Dropout
from keras.preprocessing import image
from keras.datasets import cifar10
from keras.utils import np_utils
from keras.models import Model
from keras import utils
import numpy as np


![Flowchart of Transfer Learning & Fine-Tuning](https://github.com/Hvass-Labs/TensorFlow-Tutorials/blob/master/images/10_transfer_learning_flowchart.png?raw=1)

In [16]:
from keras.datasets import cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
# From cifar10
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [17]:
x_train, val_train, y_train, val_test = train_test_split( x_train, y_train, test_size=0.2, random_state=0)
# from train_test_split
x_train.shape, val_train.shape, x_test.shape, val_test.shape

((40000, 32, 32, 3), (10000, 32, 32, 3), (10000, 32, 32, 3), (10000, 1))

In [18]:
def preprocess_input_vgg(x):

    X = np.expand_dims(x, axis=0)
    X = preprocess_input(X)
    return X[0]

In [19]:
num_classes = 10

y_train = np_utils.to_categorical(y_train, num_classes)
val_test = np_utils.to_categorical(val_test, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [20]:
datagen = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.25,
    height_shift_range=0.25, 
    horizontal_flip=True,
    preprocessing_function=preprocess_input_vgg)

evalgen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)

In [21]:
#Get back the convolutional part of a VGG network trained on ImageNet
#Notice this time we are not freezing the convolutional layers. This will allow the model to optimize the layers
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

#Create your own input format (here 3x200x200)
input = Input(shape=(32,32, 3),name = 'image_input')

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = BatchNormalization()(x)
x = Dropout(.5)(x)
x = Dense(4096, activation='relu', name='fc2')(x) #was4096
x = BatchNormalization()(x)
x = Dropout(.5)(x)
x = Dense(10, activation='softmax', name='predictions')(x)

In [27]:
#Create your own model 
my_model = Model(input, x)

epochs = 3

Adam = Adam(lr=.0001)
my_model.compile(optimizer=Adam, loss ='categorical_crossentropy', metrics=['accuracy'])

batch_size= 256
my_model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size), 
                       validation_data=datagen.flow(val_train, val_test, batch_size=batch_size), 
                       validation_steps=len(val_train)/256,
                       steps_per_epoch=len(x_train)/256, 
                       epochs=epochs)
my_model.save_weights('my_model_weights.h5')

Epoch 1/3


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


156/156 [==============================] - 34s 212ms/step - loss: 1.2440 - accuracy: 0.6094 - val_loss: 1.2784 - val_accuracy: 0.6286
Epoch 2/3
156/156 [==============================] - 32s 203ms/step - loss: 0.9475 - accuracy: 0.6893 - val_loss: 1.2139 - val_accuracy: 0.6529
Epoch 3/3
156/156 [==============================] - 32s 204ms/step - loss: 0.8167 - accuracy: 0.7279 - val_loss: 0.8446 - val_accuracy: 0.7076


In [28]:
my_model.load_weights('my_model_weights.h5')
evalgen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
score = my_model.evaluate_generator(evalgen.flow(x_test, y_test, batch_size=256), steps=len(x_test))
my_model.metrics_names , score

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  This is separate from the ipykernel package so we can avoid doing imports until


(['loss', 'accuracy'], [0.6457815766334534, 0.7815999984741211])